In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Definir o diretório de imagens para treinamento
train_dir = 'hero_pick_images'

def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 144))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

def train_model(image_dir):
    images = []
    labels = []
    class_names = list({os.path.splitext(img_file)[0] for img_file in os.listdir(image_dir) if img_file.endswith('.png')})

    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        class_name = os.path.splitext(img_file)[0]
        
        images.append(preprocess_image(img_path))
        labels.append(class_names.index(class_name))

    images = np.vstack(images)
    labels = np.array(labels)

    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 144, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(len(class_names), activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(images, labels, batch_size=32, epochs=10, validation_split=0.2)

    return model, class_names

def predict_single_image(model, img_path):
    processed_img = preprocess_image(img_path)
    predictions = model.predict(processed_img)
    return predictions

def predict_classes(predictions, class_names, threshold=0.1):
    predicted_class_indices = np.where(predictions >= threshold)[1]
    predicted_classes = [class_names[index] for index in predicted_class_indices]
    return predicted_classes

# Treinamento do modelo
model, class_names = train_model(train_dir)



Epoch 1/10
4/4 [==============================] - 10s 1s/step - loss: 5.4526 - accuracy: 0.0100 - val_loss: 7.2053 - val_accuracy: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 4s 943ms/step - loss: 1.4285 - accuracy: 0.9700 - val_loss: 8.3332 - val_accuracy: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 4s 1s/step - loss: 0.2012 - accuracy: 1.0000 - val_loss: 8.9381 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 4s 924ms/step - loss: 0.0990 - accuracy: 1.0000 - val_loss: 10.1816 - val_accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 4s 1s/step - loss: 0.0900 - accuracy: 0.9900 - val_loss: 11.9615 - val_accuracy: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 4s 954ms/step - loss: 0.1009 - accuracy: 0.9900 - val_loss: 12.7990 - val_accuracy: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 4s 929ms/step - loss: 0.0990 - accuracy: 1.0000 - val_loss: 13.2147 - val_accuracy: 0.0000e

In [12]:
def predict_classes(predictions, class_names, threshold=0.1):
    predicted_class_indices = np.where(predictions >= threshold)[1]
    predicted_classes = [class_names[index] for index in predicted_class_indices]
    return predicted_classes
# Predição em uma única imagem
predictions = predict_single_image(model, '1729987324.697981.png')

# Obtendo as classes previstas
predicted_classes = predict_classes(predictions, class_names)

print(f"As classes identificadas na imagem são: {predicted_classes}")

1/1 [==============================] - 0s 38ms/step
As classes identificadas na imagem são: ['oracle', 'magnataur', 'medusa']
